<a href="https://colab.research.google.com/github/jack978397/Colab-AI-Transcribe-Translate/blob/main/1_Whisper_Transcription_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Cell 1: 環境安裝 ---
!pip install openai-whisper yt-dlp srt "moviepy<2.0.0" Gradio google-generativeai
!apt-get install -y ffmpeg
print("\n--- ✅ API 與影音環境安裝完成！ ---")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.

--- ✅ API 與影音環境安裝完成！ ---


In [2]:
import whisper, torch, os, re, datetime, srt, yt_dlp, gc

# --- 1. 核心工具函式 (還原您的原始校正邏輯) ---
def format_srt_time(seconds):
    delta = datetime.timedelta(seconds=max(0, seconds))
    hours, remainder = divmod(delta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = delta.microseconds // 1000
    return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

def refine_srt_segments(segments, max_chars=18):
    if not segments: return []
    pre_filtered = []
    last_text = None
    for seg in segments:
        text = seg['text'].strip()
        if not text: continue
        if text == last_text and (seg['end'] - seg['start']) < 0.8:
            if pre_filtered: pre_filtered[-1]['end'] = seg['end']
            continue
        pre_filtered.append(seg)
        last_text = text

    final_segments = []
    punctuation_pattern = r'([，。、？！,.\\!\\?])'
    for segment in pre_filtered:
        text = segment['text'].strip()
        if not text or len(text) <= max_chars:
            final_segments.append(segment)
            continue
        start_time, end_time = segment['start'], segment['end']
        duration = end_time - start_time
        parts = re.split(punctuation_pattern, text)
        sub_sentences = [p for p in parts if p.strip()]
        if not sub_sentences:
            final_segments.append(segment)
            continue
        current_sentence = ""
        total_len = sum(len(s) for s in sub_sentences)
        if total_len <= 0: total_len = 1
        current_start = start_time
        for sub in sub_sentences:
            if len(current_sentence) > 0 and len(current_sentence + sub) > max_chars:
                ratio = len(current_sentence) / total_len
                new_end = current_start + (ratio * duration)
                final_segments.append({'start': current_start, 'end': new_end, 'text': current_sentence})
                current_start = new_end
                current_sentence = sub
            else:
                current_sentence += (" " if current_sentence and not re.match(r'^[，。、？！]', sub) else "") + sub
        if current_sentence:
            final_segments.append({'start': current_start, 'end': end_time, 'text': current_sentence})

    deduped = []
    for s in final_segments:
        if not deduped or s['text'] != deduped[-1]['text']:
            deduped.append(s)
        else:
            deduped[-1]['end'] = s['end']
    for i, seg in enumerate(deduped): seg['id'] = i
    return deduped

def generate_srt_from_segments(segments):
    srt_content = []
    for i, segment in enumerate(segments):
        start_time, end_time = format_srt_time(segment['start']), format_srt_time(segment['end'])
        text = segment['text'].strip()
        if text:
            srt_content.append(str(i + 1))
            srt_content.append(f"{start_time} --> {end_time}")
            srt_content.append(text)
            srt_content.append("")
    return "\n".join(srt_content)

def fix_and_process_srt(srt_content, remove_punctuation=True):
    try:
        cleaned_content = "\n".join(line for line in srt_content.strip().splitlines() if line.strip())
        subs = list(srt.parse(cleaned_content))
        for sub in subs:
            mod = sub.content.strip()
            if remove_punctuation: mod = mod.rstrip('.,。?!！?？')
            sub.content = mod.strip()
        return srt.compose(subs)
    except: return srt_content

# --- 2. 引擎模組 ---
class WhisperEngineCloud:
    def __init__(self):
        self.model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_size = None

    def load_model(self, model_size="large"):
        if self.model is not None and self.model_size == model_size: return
        self.model = whisper.load_model(model_size, device=self.device)
        self.model_size = model_size

    def transcribe(self, audio_path, prompt=""):
        return self.model.transcribe(audio_path, initial_prompt=prompt, verbose=True)

engine = WhisperEngineCloud()
print("✅ [Cell 2] 核心邏輯載入完畢")


✅ [Cell 2] 核心邏輯載入完畢


In [3]:
import gradio as gr, gc, torch, os, yt_dlp, traceback, sys, io, glob

# --- 核心：互動式取代 (同時修正 Log 文字與磁碟上的檔案) ---
def interactive_fix(current_log, old_items, new_items):
    if not old_items.strip(): return current_log, gr.update()

    olds = old_items.strip().split('\n')
    news = new_items.strip().split('\n')

    # 1. 修正介面上的日誌文字
    updated_log = current_log
    for o, n in zip(olds, news):
        o_c, n_c = o.strip(), (n.strip() if n else "")
        if o_c: updated_log = updated_log.replace(o_c, n_c)

    # 2. 同步修正磁碟上的所有 SRT 檔案，確保下載的是正確的
    srt_files = glob.glob("*.srt")
    for srt_path in srt_files:
        with open(srt_path, "r", encoding="utf-8") as f:
            content = f.read()

        new_content = content
        for o, n in zip(olds, news):
            o_c, n_c = o.strip(), (n.strip() if n else "")
            if o_c: new_content = new_content.replace(o_c, n_c)

        with open(srt_path, "w", encoding="utf-8") as f:
            f.write(new_content)

    return updated_log, srt_files # 回傳更新後的文字與檔案列表

def batch_process_transcription(files, yt_input, input_type, model_size, remove_punc, max_ch, old_txt, new_txt, download_v, progress=gr.Progress()):
    if 'engine' not in globals():
        yield "❌ 錯誤：請先執行 Cell 2！", []
        return
    output_buffer = io.StringIO(); sys.stdout = output_buffer
    log_content = "🚀 啟動批次任務...\n"

    try:
        output_files = []
        target_files = [("YouTube", yt_input)] if input_type == "YouTube 連結" else [("Local", f.name) for f in files]

        for i, (f_type, f_path) in enumerate(target_files):
            filename = os.path.basename(f_path) if f_type == "Local" else "YouTube 影片"
            log_content += f"正在處理 ({i+1}/{len(target_files)}): {filename}...\n"; yield log_content, output_files

            # --- 下載/準備 ---
            temp_audio = 'cloud_temp_audio.mp3'
            if f_type == "YouTube":
                if download_v:
                    ydl_opts = {'format': 'mp4', 'outtmpl': 'yt_video_%(title)s.%(ext)s', 'quiet': True}
                    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                        info = ydl.extract_info(f_path, download=True); v_path = ydl.prepare_filename(info); filename = info.get('title', 'YouTube影片')
                    os.system(f'ffmpeg -i "{v_path}" -q:a 0 -map a "{temp_audio}" -y -loglevel error')
                    output_files.append(v_path)
                else:
                    ydl_opts = {'format': 'bestaudio', 'outtmpl': 'tmp.m4a', 'quiet': True}
                    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                        info = ydl.extract_info(f_path, download=True); d_file = ydl.prepare_filename(info); filename = info.get('title', 'YouTube影片')
                    os.system(f'ffmpeg -i "{d_file}" "{temp_audio}" -y -loglevel error')
                audio_source = temp_audio
            else:
                os.system(f'ffmpeg -i "{f_path}" "{temp_audio}" -y -loglevel error'); audio_source = temp_audio

            # --- 轉錄 ---
            gc.collect(); torch.cuda.empty_cache() if torch.cuda.is_available() else None
            engine.load_model(model_size); result = engine.transcribe(audio_source)
            log_content += output_buffer.getvalue(); output_buffer.truncate(0); output_buffer.seek(0)

            # --- 校正與初步取代 ---
            refined = refine_srt_segments(result['segments'], max_chars=max_ch)
            final_srt = generate_srt_from_segments(refined)
            final_srt = fix_and_process_srt(final_srt, remove_punctuation=remove_punc)

            # 這裡也會跑一次預設取代
            olds, news = old_txt.strip().split('\n'), new_txt.strip().split('\n')
            for o, n in zip(olds, news):
                if o.strip(): final_srt = final_srt.replace(o.strip(), n.strip() if n else "")

            # --- 存檔 ---
            safe_name = "".join([c if c.isalnum() else "_" for c in filename[:15]])
            out_path = os.path.abspath(f"transcript_{safe_name}.srt")
            with open(out_path, "w", encoding="utf-8") as f: f.write(final_srt)
            output_files.append(out_path)
            log_content += f"✅ {filename} 完成！\n"; yield log_content, output_files

        log_content += "\n--- 🎉 全部檔案辨識完畢 ---"; yield log_content, output_files
    except Exception as e: yield log_content + f"\n❌ 錯誤: {e}", output_files
    finally: sys.stdout = sys.__stdout__

# --- UI 佈置 ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🚀 Whisper Pro 影音批次轉錄中心")
    with gr.Row():
        with gr.Column(scale=1):
            t_mode = gr.Radio(["檔案上傳 (支援多選)", "YouTube 連結"], label="輸入模式", value="檔案上傳 (支援多選)")
            t_files = gr.File(label="上傳區", file_count="multiple", visible=True)
            t_url = gr.Textbox(label="YouTube 網址", visible=False)
            t_model = gr.Dropdown(["base", "medium", "large"], label="使用模型", value="large")
            t_punc = gr.Checkbox(label="自動移除句尾標點", value=True)

            with gr.Accordion("⚙️ 進階格式與取代設定", open=True):
                t_max_ch = gr.Slider(10, 30, value=18, step=1, label="每行最大字元")
                t_dv = gr.Checkbox(label="同時下載 YouTube 影片 (mp4)", value=False)
                with gr.Row():
                    t_old = gr.Textbox(label="舊詞清單 (每行一個)", lines=3)
                    t_new = gr.Textbox(label="新詞清單 (對應左方)", lines=3)
                # ✨ 新增按鈕：手動補強取代
                t_replace_btn = gr.Button("🪄 立即執行修正 (同時更新日誌與檔案)", variant="secondary")

            t_btn = gr.Button("🔥 啟動完整辨識任務", variant="primary")
        with gr.Column(scale=2):
            t_log = gr.Textbox(label="處理進度與日誌 (可手動修改內容後按下修正按鈕)", lines=18)
            t_out = gr.File(label="📥 轉錄成果下載 (SRT 檔案列表)", file_count="multiple")

    t_mode.change(lambda x: (gr.update(visible=x=="檔案上傳 (支援多選)"), gr.update(visible=x=="YouTube 連結")), t_mode, [t_files, t_url])

    # 事件：啟動辨識
    t_btn.click(batch_process_transcription, [t_files, t_url, t_mode, t_model, t_punc, t_max_ch, t_old, t_new, t_dv], [t_log, t_out])

    # ✨ 事件：手動取代 (點了之後，日誌會變、磁碟裡的檔案也會更新，下載連結會同步變動)
    t_replace_btn.click(interactive_fix, inputs=[t_log, t_old, t_new], outputs=[t_log, t_out])

demo.queue().launch(share=True, debug=True)


/tmp/ipython-input-1333738190.py:92: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9ed98ecee7bf4310fe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
